##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [1]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
tf.keras.backend.set_floatx('float64')
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])


For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.63420635, -0.37837925, -0.08310245,  0.78474207, -0.72798594,
         0.86291529, -0.5216555 , -0.22003263,  0.54920403,  0.22595697]])

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.14597282, 0.05302878, 0.07124407, 0.1696871 , 0.03738345,
        0.18348435, 0.04595022, 0.06212705, 0.13407752, 0.09704464]])

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

1.695625901222229

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [10]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 66us/sample - loss: 0.2988 - accuracy: 0.9133
Epoch 2/5
60000/60000 [==============================] - 4s 60us/sample - loss: 0.1456 - accuracy: 0.9572
Epoch 3/5
60000/60000 [==============================] - 4s 62us/sample - loss: 0.1101 - accuracy: 0.9665
Epoch 4/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0890 - accuracy: 0.9728
Epoch 5/5
60000/60000 [==============================] - 3s 57us/sample - loss: 0.0760 - accuracy: 0.9759


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 0s - loss: 0.0366 - accuracy: 0.9780


[0.06972638740955825, 0.978]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: id=30042, shape=(5, 10), dtype=float64, numpy=
array([[2.91918971e-07, 9.21563324e-07, 1.94812329e-05, 1.56600502e-03,
        2.35425593e-10, 4.88805419e-07, 2.76605730e-13, 9.98366306e-01,
        5.39349454e-06, 4.11115129e-05],
       [3.68631115e-06, 2.06172118e-04, 9.99684798e-01, 9.20654423e-05,
        2.89966351e-12, 5.29673749e-06, 1.94003960e-06, 4.43612708e-12,
        6.04177846e-06, 6.49269370e-13],
       [4.58623883e-07, 9.99472926e-01, 3.29991755e-05, 7.10204203e-06,
        2.64073793e-05, 5.65089280e-06, 4.46128761e-05, 1.67355691e-04,
        2.42034262e-04, 4.53246279e-07],
       [9.99845068e-01, 8.27275975e-10, 2.52981243e-05, 7.13861564e-07,
        2.21149625e-07, 4.49175468e-06, 1.18324977e-04, 3.12540747e-06,
        1.51181692e-08, 2.74095909e-06],
       [2.37192690e-05, 1.79650031e-09, 1.31433356e-05, 2.16355135e-08,
        9.97212084e-01, 3.77444104e-07, 4.07800485e-05, 1.40111722e-04,
        5.59433487e-06, 2.56416683e-03]])>